## Linear Classifier in TensorFlow 
Using Low Level API in Eager Execution mode

### Load tensorflow

In [1]:
import tensorflow as tf

C:\Users\msrikanta\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
#Enable Eager Execution if using tensflow version < 2.0
#From tensorflow v2.0 onwards, Eager Execution will be enabled by default
#tf.enable_eager_execution()

### Collect Data

In [3]:
#from google.colab import drive
#drive.mount('/gdrive')

In [4]:
import pandas as pd
import numpy as np

In [5]:
data = pd.read_csv('prices.csv')

### Check all columns in the dataset

In [6]:
data.shape

(851264, 7)

In [7]:
data.head()

,date,symbol,open,close,low,high,volume
0,2016-01-05 00:00:00,WLTW,123.430000,125.839996,122.309998,126.250000,2163600.0
1,2016-01-06 00:00:00,WLTW,125.239998,119.980003,119.940002,125.540001,2386400.0
2,2016-01-07 00:00:00,WLTW,116.379997,114.949997,114.930000,119.739998,2489500.0
3,2016-01-08 00:00:00,WLTW,115.480003,116.620003,113.500000,117.440002,2006300.0
4,2016-01-11 00:00:00,WLTW,117.010002,114.970001,114.089996,117.330002,1408600.0


In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 851264 entries, 0 to 851263
Data columns (total 7 columns):
date      851264 non-null object
symbol    851264 non-null object
open      851264 non-null float64
close     851264 non-null float64
low       851264 non-null float64
high      851264 non-null float64
volume    851264 non-null float64
dtypes: float64(5), object(2)
memory usage: 45.5+ MB


### Drop columns `date` and  `symbol`

In [9]:
data = data.drop(['date','symbol'],axis=1)

In [10]:
data.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


### Consider only first 1000 rows in the dataset for building feature set and target set
Target 'Volume' has very high values. Divide 'Volume' by 1000,000

In [11]:
data_sample = data.head(1000)

In [12]:
data_sample.shape

(1000, 5)

In [13]:
data_sample['volume'] = data_sample['volume'] / 1000000

C:\Users\msrikanta\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [14]:
data_sample.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2.1636
1,125.239998,119.980003,119.940002,125.540001,2.3864
2,116.379997,114.949997,114.930000,119.739998,2.4895
3,115.480003,116.620003,113.500000,117.440002,2.0063
4,117.010002,114.970001,114.089996,117.330002,1.4086


### Divide the data into train and test sets

In [15]:
from sklearn.model_selection import train_test_split

In [16]:
X = data_sample.drop('volume',axis=1)

In [17]:
X.shape

(1000, 4)

In [18]:
Y = data_sample['volume']

In [19]:
Y.shape

(1000,)

#### Convert Training and Test Data to numpy float32 arrays


In [20]:
import numpy as np
X_arr = X.values

In [21]:
Y_arr = Y.values

In [22]:
type(X_arr)

numpy.ndarray

In [23]:
type(Y_arr)

numpy.ndarray

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X_arr, Y_arr, test_size=0.30, random_state=7)

### Normalize the data
You can use Normalizer from sklearn.preprocessing

In [25]:
from sklearn.preprocessing import Normalizer

In [26]:
Norm = Normalizer()
X_train_n = Norm.fit_transform(X_train)
X_test_n = Norm.fit_transform(X_test) 

## Building the Model in tensorflow

1.Define Weights and Bias, use tf.zeros to initialize weights and Bias

In [27]:
#W = tf.Variable(tf.zeros(shape=[4,1]), name="Weights")
#b = tf.Variable(tf.zeros(shape=[1]),name="Bias")

In [28]:
W = tf.random_normal([4,1])
b = tf.random_normal([1])

2.Define a function to calculate prediction

In [29]:
X_train_n = np.float32(X_train_n)
X_test_n = np.float32(X_test_n) 

In [30]:
#y = tf.add(tf.matmul(X_train_n,W),b,name='output')
def prediction(x, w, b):
    xw_matmul = tf.matmul(x, w)
    y = tf.add(xw_matmul, b)
    return y

3.Loss (Cost) Function [Mean square error]

In [31]:
#loss = tf.reduce_mean(tf.square(y-y_train),name='Loss')
def loss(y_actual, y_predicted):
    diff = y_actual - y_predicted
    sqr = tf.square(diff)
    avg = tf.reduce_mean(sqr)
    return avg

##### Gradient function

4.Function to train the Model

1.   Record all the mathematical steps to calculate Loss
2.   Calculate Gradients of Loss w.r.t weights and bias
3.   Update Weights and Bias based on gradients and learning rate to minimize loss

In [32]:
#train_op = tf.train.GradientDescentOptimizer(0.03).minimize(loss)
def train(x, y_actual, w, b, learning_rate=0.03):    
    #Record math ops on 'tape' to calculate loss
    with tf.GradientTape() as t:
        t.watch([w,b])    
        current_prediction = prediction(x, w, b)
        current_loss = loss(y_actual, current_prediction)
    #Calculate Gradients for Loss w.r.t Weights and Bias
    dw, db = t.gradient(current_loss,[w, b])
    #Update Weights and Bias
    w = w - learning_rate*dw
    b = b - learning_rate*db
    return w, b

## Train the model for 100 epochs 
1. Observe the training loss at every iteration
2. Observe Test loss at every 5th iteration

In [34]:
#Training the model for 100 iterations
for i in range(100):
    W, b = train(X_train_n, y_train, W, b)
    print('W=',W,'b=',b)
    #print('Current Loss on iteration', i, loss(y_train, prediction(X_train_n, W, b)).numpy()) -- use this with #tf.enable_eager_execution()
    print('Current Loss on iteration', i, loss(y_train, prediction(X_train_n, W, b)))

W= Tensor("sub_4:0", shape=(4, 1), dtype=float32) b= Tensor("sub_5:0", shape=(1,), dtype=float32)
Current Loss on iteration 0 Tensor("Mean_2:0", shape=(), dtype=float32)
W= Tensor("sub_8:0", shape=(4, 1), dtype=float32) b= Tensor("sub_9:0", shape=(1,), dtype=float32)
Current Loss on iteration 1 Tensor("Mean_4:0", shape=(), dtype=float32)
W= Tensor("sub_12:0", shape=(4, 1), dtype=float32) b= Tensor("sub_13:0", shape=(1,), dtype=float32)
Current Loss on iteration 2 Tensor("Mean_6:0", shape=(), dtype=float32)
W= Tensor("sub_16:0", shape=(4, 1), dtype=float32) b= Tensor("sub_17:0", shape=(1,), dtype=float32)
Current Loss on iteration 3 Tensor("Mean_8:0", shape=(), dtype=float32)
W= Tensor("sub_20:0", shape=(4, 1), dtype=float32) b= Tensor("sub_21:0", shape=(1,), dtype=float32)
Current Loss on iteration 4 Tensor("Mean_10:0", shape=(), dtype=float32)
W= Tensor("sub_24:0", shape=(4, 1), dtype=float32) b= Tensor("sub_25:0", shape=(1,), dtype=float32)
Current Loss on iteration 5 Tensor("Mean_12

W= Tensor("sub_200:0", shape=(4, 1), dtype=float32) b= Tensor("sub_201:0", shape=(1,), dtype=float32)
Current Loss on iteration 49 Tensor("Mean_100:0", shape=(), dtype=float32)
W= Tensor("sub_204:0", shape=(4, 1), dtype=float32) b= Tensor("sub_205:0", shape=(1,), dtype=float32)
Current Loss on iteration 50 Tensor("Mean_102:0", shape=(), dtype=float32)
W= Tensor("sub_208:0", shape=(4, 1), dtype=float32) b= Tensor("sub_209:0", shape=(1,), dtype=float32)
Current Loss on iteration 51 Tensor("Mean_104:0", shape=(), dtype=float32)
W= Tensor("sub_212:0", shape=(4, 1), dtype=float32) b= Tensor("sub_213:0", shape=(1,), dtype=float32)
Current Loss on iteration 52 Tensor("Mean_106:0", shape=(), dtype=float32)
W= Tensor("sub_216:0", shape=(4, 1), dtype=float32) b= Tensor("sub_217:0", shape=(1,), dtype=float32)
Current Loss on iteration 53 Tensor("Mean_108:0", shape=(), dtype=float32)
W= Tensor("sub_220:0", shape=(4, 1), dtype=float32) b= Tensor("sub_221:0", shape=(1,), dtype=float32)
Current Loss 

W= Tensor("sub_400:0", shape=(4, 1), dtype=float32) b= Tensor("sub_401:0", shape=(1,), dtype=float32)
Current Loss on iteration 99 Tensor("Mean_200:0", shape=(), dtype=float32)


### Get the shapes and values of W and b

In [35]:
print('W=',W,'b=',b)

W= Tensor("sub_400:0", shape=(4, 1), dtype=float32) b= Tensor("sub_401:0", shape=(1,), dtype=float32)


### Model Prediction on 1st Examples in Test Dataset

In [38]:
for i in range(100):
    W, b = train(X_test_n, y_test, W, b)
    print('W=',W,'b=',b)
    print('Current Loss on iteration', i, loss(y_test, prediction(X_test_n, W, b)))

W= Tensor("sub_411:0", shape=(4, 1), dtype=float32) b= Tensor("sub_412:0", shape=(1,), dtype=float32)
Current Loss on iteration 0 Tensor("Mean_205:0", shape=(), dtype=float32)
W= Tensor("sub_415:0", shape=(4, 1), dtype=float32) b= Tensor("sub_416:0", shape=(1,), dtype=float32)
Current Loss on iteration 1 Tensor("Mean_207:0", shape=(), dtype=float32)
W= Tensor("sub_419:0", shape=(4, 1), dtype=float32) b= Tensor("sub_420:0", shape=(1,), dtype=float32)
Current Loss on iteration 2 Tensor("Mean_209:0", shape=(), dtype=float32)
W= Tensor("sub_423:0", shape=(4, 1), dtype=float32) b= Tensor("sub_424:0", shape=(1,), dtype=float32)
Current Loss on iteration 3 Tensor("Mean_211:0", shape=(), dtype=float32)
W= Tensor("sub_427:0", shape=(4, 1), dtype=float32) b= Tensor("sub_428:0", shape=(1,), dtype=float32)
Current Loss on iteration 4 Tensor("Mean_213:0", shape=(), dtype=float32)
W= Tensor("sub_431:0", shape=(4, 1), dtype=float32) b= Tensor("sub_432:0", shape=(1,), dtype=float32)
Current Loss on it

W= Tensor("sub_615:0", shape=(4, 1), dtype=float32) b= Tensor("sub_616:0", shape=(1,), dtype=float32)
Current Loss on iteration 51 Tensor("Mean_307:0", shape=(), dtype=float32)
W= Tensor("sub_619:0", shape=(4, 1), dtype=float32) b= Tensor("sub_620:0", shape=(1,), dtype=float32)
Current Loss on iteration 52 Tensor("Mean_309:0", shape=(), dtype=float32)
W= Tensor("sub_623:0", shape=(4, 1), dtype=float32) b= Tensor("sub_624:0", shape=(1,), dtype=float32)
Current Loss on iteration 53 Tensor("Mean_311:0", shape=(), dtype=float32)
W= Tensor("sub_627:0", shape=(4, 1), dtype=float32) b= Tensor("sub_628:0", shape=(1,), dtype=float32)
Current Loss on iteration 54 Tensor("Mean_313:0", shape=(), dtype=float32)
W= Tensor("sub_631:0", shape=(4, 1), dtype=float32) b= Tensor("sub_632:0", shape=(1,), dtype=float32)
Current Loss on iteration 55 Tensor("Mean_315:0", shape=(), dtype=float32)
W= Tensor("sub_635:0", shape=(4, 1), dtype=float32) b= Tensor("sub_636:0", shape=(1,), dtype=float32)
Current Loss 

W= Tensor("sub_807:0", shape=(4, 1), dtype=float32) b= Tensor("sub_808:0", shape=(1,), dtype=float32)
Current Loss on iteration 99 Tensor("Mean_403:0", shape=(), dtype=float32)


## Classification using tf.Keras

In this exercise, we will build a Deep Neural Network using tf.Keras. We will use Iris Dataset for this exercise.

### Load tensorflow if not done already

### Load the given Iris data using pandas (Iris.csv)

In [39]:
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


In [40]:
iris = pd.read_csv('Iris.csv')

In [41]:
iris.shape

(150, 6)

In [42]:
iris.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [43]:
iris.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 6 columns):
Id               150 non-null int64
SepalLengthCm    150 non-null float64
SepalWidthCm     150 non-null float64
PetalLengthCm    150 non-null float64
PetalWidthCm     150 non-null float64
Species          150 non-null object
dtypes: float64(4), int64(1), object(1)
memory usage: 7.1+ KB


### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

In [44]:
iris_oh = pd.get_dummies(iris,columns = ["Species"],drop_first = False)

In [45]:
iris_oh

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species_Iris-setosa,Species_Iris-versicolor,Species_Iris-virginica
0,1,5.1,3.5,1.4,0.2,1,0,0
1,2,4.9,3.0,1.4,0.2,1,0,0
2,3,4.7,3.2,1.3,0.2,1,0,0
3,4,4.6,3.1,1.5,0.2,1,0,0
4,5,5.0,3.6,1.4,0.2,1,0,0
5,6,5.4,3.9,1.7,0.4,1,0,0
6,7,4.6,3.4,1.4,0.3,1,0,0
7,8,5.0,3.4,1.5,0.2,1,0,0
8,9,4.4,2.9,1.4,0.2,1,0,0
9,10,4.9,3.1,1.5,0.1,1,0,0


In [46]:
iris_oh.shape

(150, 8)

In [47]:
iris_oh.to_csv("iris_oh.csv")

### Splitting the data into feature set and target set

In [48]:
X = iris_oh.drop(['Id','Species_Iris-setosa','Species_Iris-versicolor','Species_Iris-virginica'],axis=1)

In [49]:
Y = iris_oh.filter(["Species_Iris-setosa","Species_Iris-versicolor","Species_Iris-virginica"], axis=1)

In [50]:
X.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [51]:
X.shape

(150, 4)

In [52]:
Y.head()

,Species_Iris-setosa,Species_Iris-versicolor,Species_Iris-virginica
0,1,0,0
1,1,0,0
2,1,0,0
3,1,0,0
4,1,0,0


In [53]:
Y.shape

(150, 3)

### Divide the dataset into Training and test (70:30)

In [54]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.30, random_state=7)

In [55]:
X_train.shape

(105, 4)

In [56]:
X_test.shape

(45, 4)

In [57]:
y_train.shape

(105, 3)

In [58]:
y_test.shape

(45, 3)

###  Building Model in tf.keras

Build a Linear Classifier model  <br>
1.  Use Dense Layer  with input shape of 4 (according to the feature set) and number of outputs set to 3<br> 
2. Apply Softmax on Dense Layer outputs <br>
3. Use SGD as Optimizer
4. Use categorical_crossentropy as loss function 

In [91]:
model = Sequential()
model.add(Dense(8, input_dim=4, activation='relu'))
model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

### Model Training 

In [92]:
model.fit(X_train, y_train, epochs=150, batch_size=5)

Epoch 1/150
105/105 [==============================] - 1s 9ms/step - loss: 1.2757 - acc: 0.3429
Epoch 2/150
105/105 [==============================] - 0s 324us/step - loss: 0.8316 - acc: 0.6571
Epoch 3/150
105/105 [==============================] - 0s 314us/step - loss: 0.6970 - acc: 0.6952
Epoch 4/150
105/105 [==============================] - 0s 314us/step - loss: 0.6285 - acc: 0.7238
Epoch 5/150
105/105 [==============================] - 0s 381us/step - loss: 0.5764 - acc: 0.7429
Epoch 6/150
105/105 [==============================] - 0s 314us/step - loss: 0.5577 - acc: 0.6667
Epoch 7/150
105/105 [==============================] - 0s 314us/step - loss: 0.5276 - acc: 0.7333
Epoch 8/150
105/105 [==============================] - 0s 371us/step - loss: 0.4983 - acc: 0.7905
Epoch 9/150
105/105 [==============================] - 0s 257us/step - loss: 0.4950 - acc: 0.7524
Epoch 10/150
105/105 [==============================] - 0s 333us/step - loss: 0.4827 - acc: 0.7333
Epoch 11/150
105/105 

105/105 [==============================] - 0s 276us/step - loss: 0.1566 - acc: 0.9429
Epoch 84/150
105/105 [==============================] - 0s 390us/step - loss: 0.1510 - acc: 0.9524
Epoch 85/150
105/105 [==============================] - 0s 257us/step - loss: 0.1523 - acc: 0.9524
Epoch 86/150
105/105 [==============================] - 0s 362us/step - loss: 0.1476 - acc: 0.9619
Epoch 87/150
105/105 [==============================] - 0s 371us/step - loss: 0.1568 - acc: 0.9524
Epoch 88/150
105/105 [==============================] - 0s 314us/step - loss: 0.1385 - acc: 0.9619
Epoch 89/150
105/105 [==============================] - 0s 352us/step - loss: 0.1399 - acc: 0.9619
Epoch 90/150
105/105 [==============================] - 0s 333us/step - loss: 0.1470 - acc: 0.9429
Epoch 91/150
105/105 [==============================] - 0s 352us/step - loss: 0.1368 - acc: 0.9810
Epoch 92/150
105/105 [==============================] - 0s 352us/step - loss: 0.1341 - acc: 0.9714
Epoch 93/150
105/105 [=

### Model Prediction

In [93]:
scores = model.evaluate(X_test, y_test)
scores[1]*100

45/45 [==============================] - 0s 7ms/step


93.33333333333333

### Save the Model

In [94]:
model.save("iris_keras.h5")

### Build and Train a Deep Neural network with 2 hidden layer  - Optional - For Practice

Does it perform better than Linear Classifier? What could be the reason for difference in performance?

In [95]:
model = Sequential()
model.add(Dense(8, input_dim=4, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

In [96]:
model.fit(X_train, y_train, epochs=150, batch_size=10)

Epoch 1/150
105/105 [==============================] - 1s 8ms/step - loss: 1.0508 - acc: 0.3619
Epoch 2/150
105/105 [==============================] - 0s 171us/step - loss: 0.9554 - acc: 0.4381
Epoch 3/150
105/105 [==============================] - 0s 228us/step - loss: 0.8955 - acc: 0.4857
Epoch 4/150
105/105 [==============================] - 0s 181us/step - loss: 0.8269 - acc: 0.6952
Epoch 5/150
105/105 [==============================] - 0s 181us/step - loss: 0.7731 - acc: 0.8190
Epoch 6/150
105/105 [==============================] - 0s 209us/step - loss: 0.7297 - acc: 0.7619
Epoch 7/150
105/105 [==============================] - 0s 162us/step - loss: 0.6941 - acc: 0.7714
Epoch 8/150
105/105 [==============================] - 0s 228us/step - loss: 0.6332 - acc: 0.8667
Epoch 9/150
105/105 [==============================] - 0s 219us/step - loss: 0.5935 - acc: 0.7905
Epoch 10/150
105/105 [==============================] - 0s 171us/step - loss: 0.5539 - acc: 0.9238
Epoch 11/150
105/105 

105/105 [==============================] - 0s 238us/step - loss: 0.1165 - acc: 0.9524
Epoch 84/150
105/105 [==============================] - 0s 181us/step - loss: 0.1138 - acc: 0.9429
Epoch 85/150
105/105 [==============================] - 0s 200us/step - loss: 0.1288 - acc: 0.9524
Epoch 86/150
105/105 [==============================] - 0s 228us/step - loss: 0.1161 - acc: 0.9619
Epoch 87/150
105/105 [==============================] - 0s 181us/step - loss: 0.0991 - acc: 0.9714
Epoch 88/150
105/105 [==============================] - 0s 162us/step - loss: 0.3450 - acc: 0.8571
Epoch 89/150
105/105 [==============================] - 0s 247us/step - loss: 0.1362 - acc: 0.9619
Epoch 90/150
105/105 [==============================] - 0s 181us/step - loss: 0.1028 - acc: 0.9619
Epoch 91/150
105/105 [==============================] - 0s 247us/step - loss: 0.1115 - acc: 0.9524
Epoch 92/150
105/105 [==============================] - 0s 238us/step - loss: 0.1090 - acc: 0.9429
Epoch 93/150
105/105 [=

In [97]:
scores = model.evaluate(X_test, y_test)
scores[1]*100

45/45 [==============================] - 0s 8ms/step


97.77777777777777

In [73]:
# Yes additional hidden layers is improving the performance due to additional neurons